---
cdt: 2024-09-09T16:17:32
title: Separation of CUPRAC and Raw Signal Tables
description: The CUPRAC and Raw data is in one table which is too large for efficient querying. Need to seperate them
project: dataset_EDA
---

As we have found throughout the project, querying on the combined long `chromatogram_spectra_long` table is slow. Considering that CUPRAC and raw are fundamentally different datasets, there is no reason to keep them in one table that outwieghs the negatives. Thus they will be seperated into two.

Considering that there will be associated tables downtrack - peak tables, possibly baseline corrected, etc. We should make schemas for each. 'raw', and 'cuprac'


In [4]:
%reload_ext autoreload
%autoreload 2

import duckdb as db
import polars as pl
from pca_analysis.experiments.constants import db_path

con = db.connect(db_path)


In [5]:
con.sql(
"""--sql
CREATE SCHEMA IF NOT EXISTS raw;
CREATE SCHEMA IF NOT EXISTS cuprac;
"""
)


create cuprac table first.

In [10]:
con.sql(
"""--sql
CREATE OR REPLACE TABLE cuprac.cs_long AS (
    WITH 
        labeled_data AS (
            SELECT
                *
            FROM
                pbl.chromatogram_spectra_long
            JOIN
                (select id, sample_num, detection FROM pbl.sample_metadata) as mta
            USING
                (id)
        ),
        cuprac_data AS (
            SELECT
                *
            FROM
                labeled_data
            WHERE
                detection = 'cuprac'
        )
    SELECT
        *
    FROM
        cuprac_data
);
SELECT
    *
FROM
    cuprac.cs_long
ORDER BY
    sample_num, wavelength, idx
"""
).pl()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

idx,id,mins,wavelength,absorbance,sample_num,detection
i64,str,f64,i32,f64,i64,str
0,"""87040513-0579-4793-9593-6e3322…",0.002083,190,3.626853,105,"""cuprac"""
1,"""87040513-0579-4793-9593-6e3322…",0.00875,190,5.647995,105,"""cuprac"""
2,"""87040513-0579-4793-9593-6e3322…",0.015417,190,7.061757,105,"""cuprac"""
3,"""87040513-0579-4793-9593-6e3322…",0.022083,190,7.529549,105,"""cuprac"""
4,"""87040513-0579-4793-9593-6e3322…",0.02875,190,7.070497,105,"""cuprac"""
…,…,…,…,…,…,…
5995,"""79177dc7-9f72-40c6-8ea7-0661a7…",39.972917,600,0.06628,175,"""cuprac"""
5996,"""79177dc7-9f72-40c6-8ea7-0661a7…",39.979583,600,0.102244,175,"""cuprac"""
5997,"""79177dc7-9f72-40c6-8ea7-0661a7…",39.98625,600,0.124872,175,"""cuprac"""


now do the same for raw.

In [14]:
con.sql(
"""--sql
CREATE OR REPLACE TABLE raw.cs_long AS (
    WITH 
        labeled_data AS (
            SELECT
                *
            FROM
                pbl.chromatogram_spectra_long
            JOIN
                (select id, sample_num, detection FROM pbl.sample_metadata) as mta
            USING
                (id)
        ),
        raw_data AS (
            SELECT
                *
            FROM
                labeled_data
            WHERE
                detection = 'raw'
        )
    SELECT
        *
    FROM
        raw_data
);
SELECT
    *
FROM
    raw.cs_long
ORDER BY
    sample_num, wavelength, idx
"""
).pl()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

idx,id,mins,wavelength,absorbance,sample_num,detection
i64,str,f64,i32,f64,i64,str
0,"""4fe49506-74e4-473b-b7a8-23500c…",0.005417,190,0.112936,1,"""raw"""
1,"""4fe49506-74e4-473b-b7a8-23500c…",0.012083,190,0.13493,1,"""raw"""
2,"""4fe49506-74e4-473b-b7a8-23500c…",0.01875,190,0.126094,1,"""raw"""
3,"""4fe49506-74e4-473b-b7a8-23500c…",0.025417,190,0.077926,1,"""raw"""
4,"""4fe49506-74e4-473b-b7a8-23500c…",0.032083,190,-0.004746,1,"""raw"""
…,…,…,…,…,…,…
7795,"""afb7db75-a452-4a65-976d-379f93…",51.971867,400,0.004008,104,"""raw"""
7796,"""afb7db75-a452-4a65-976d-379f93…",51.978533,400,0.00491,104,"""raw"""
7797,"""afb7db75-a452-4a65-976d-379f93…",51.9852,400,0.006206,104,"""raw"""


Done.